In [ ]:
!pip install boto3

In [95]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kinesis-asl_2.12/3.1.2/spark-streaming-kinesis-asl_2.12-3.1.2.jar,https://repo1.maven.org/maven2/com/amazonaws/amazon-kinesis-client/1.14.8/amazon-kinesis-client-1.14.8.jar,https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-kinesis/1.12.31/aws-java-sdk-kinesis-1.12.31.jar,/home/jovyan/.local/lib/python3.7/site-packages pyspark-shell'

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.types import StructField, StructType, StringType, IntegerType
from pyspark.streaming.kinesis import KinesisUtils, InitialPositionInStream
import boto3,json

In [96]:
# make sure you have your AWS cerdentials set before you can run this

# creating the Kinesis stream
client = boto3.client('kinesis')
stream_name='pyspark-kinesis'

try:
    print("create a new stream")
    client.create_stream(
            StreamName=stream_name,
            ShardCount=1)
except:
    print("the stream exists")

create a new stream
the stream exists


In [97]:
# creating a couple of messages to send to kinesis
messages = [
    {'message_type': 'message1', 'count': 2},
    {'message_type': 'message2', 'count': 1},
    {'message_type': 'message1', 'count': 2},
    {'message_type': 'message3', 'count': 3},
    {'message_type': 'message1', 'count': 5}
]

In [98]:
for message in messages:
    client.put_record(
        StreamName=stream_name,
        Data=json.dumps(message),
        PartitionKey='part_key')

In [99]:
sc = SparkContext(appName="PythonStreamingKinesisWordCountAsl")
ssc = StreamingContext(sc, 1)

In [100]:
kinesis = KinesisUtils.createStream(
    ssc, stream_name,stream_name, 'https://kinesis.us-east-1.amazonaws.com','us-east-1', \
    InitialPositionInStream.TRIM_HORIZON, 2)

In [92]:
def format_sample(x):
    data = json.loads(x)
    return (data[0], json.dumps(data))

In [101]:
parsed = kinesis.map(lambda x: format_sample(x))
parsed.pprint()

In [102]:
ssc.start()
ssc.awaitTerminationOrTimeout(5)

-------------------------------------------
Time: 2022-03-21 19:05:15
-------------------------------------------

-------------------------------------------
Time: 2022-03-21 19:05:16
-------------------------------------------

-------------------------------------------
Time: 2022-03-21 19:05:17
-------------------------------------------

-------------------------------------------
Time: 2022-03-21 19:05:18
-------------------------------------------

-------------------------------------------
Time: 2022-03-21 19:05:19
-------------------------------------------



In [ ]:
# cleanup
# client.delete_stream(StreamName=stream_name)